In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import urllib
from shapely.geometry import Point
import geopy.distance
from geopy import distance
from geopy import Point
import matplotlib.pyplot as plt 
from scipy import stats
import seaborn as sns
import geocoder
import json
import urllib
import urllib.request
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelBinarizer

The Point of this notebook is to look at which attribute is the best predictor for the type of bikes

In [2]:
bikes=pd.read_csv('bikes_full_SF_Feb_ml.csv', error_bad_lines=False)
elevation=pd.read_csv('elevation_sf.csv', error_bad_lines=False)

In [3]:
elevation.set_index('station_id',inplace=True)
elevation

,SF-M26,SF-M25,SF-M26-2,SF-L27,SF-K26,SF-J27,SF-M30,SF-L27-2,SF-J29-3,SF-J29,...,SF-Z7,SF-Z6,SF-W8,SF-BB17,SF-AA15,SF-Z16,SF-Y14,SF-X14,SF-X19,SF-Y25
station_id,,,,,,,,,,,,,,,,,,,,,
SF-M26,0.0,3.0,-8.0,-8.0,-8.0,-8.0,-8.0,-8.0,-8.0,-8.0,...,28.0,19.0,50.0,52.0,59.0,44.0,87.0,90.0,25.0,15.0
SF-M25,-3.0,0.0,-11.0,-11.0,-11.0,-11.0,-11.0,-11.0,-11.0,-11.0,...,25.0,16.0,47.0,49.0,56.0,41.0,84.0,87.0,22.0,12.0
SF-M26-2,8.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,36.0,27.0,58.0,60.0,67.0,52.0,95.0,98.0,33.0,23.0
SF-L27,8.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,36.0,27.0,58.0,60.0,67.0,52.0,95.0,98.0,33.0,23.0
SF-K26,8.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,36.0,27.0,58.0,60.0,67.0,52.0,95.0,98.0,33.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SF-Z16,-44.0,-41.0,-52.0,-52.0,-52.0,-52.0,-52.0,-52.0,-52.0,-52.0,...,-16.0,-25.0,6.0,8.0,15.0,0.0,43.0,46.0,-19.0,-29.0
SF-Y14,-87.0,-84.0,-95.0,-95.0,-95.0,-95.0,-95.0,-95.0,-95.0,-95.0,...,-59.0,-68.0,-37.0,-35.0,-28.0,-43.0,0.0,3.0,-62.0,-72.0
SF-X14,-90.0,-87.0,-98.0,-98.0,-98.0,-98.0,-98.0,-98.0,-98.0,-98.0,...,-62.0,-71.0,-40.0,-38.0,-31.0,-46.0,-3.0,0.0,-65.0,-75.0


In [4]:
bikes['started_at']=pd.to_datetime(bikes['started_at'],errors='coerce')
bikes['ended_at']=pd.to_datetime(bikes['ended_at'],errors='coerce')
bikes['start_hour']=bikes['started_at'].dt.hour
bikes['end_hour']=bikes['ended_at'].dt.hour
bikes

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,displacement,duration,velocity,start_hour,end_hour
0,072879C2ED883CF1,classic_bike,2021-02-12 19:08:22,2021-02-12 19:15:02,Townsend St at 5th St,SF-K28,Mississippi St at 17th St,SF-N29,37.775214,-122.397477,37.764793,-122.394796,member,1.180490e+06,400.0,2951.224312,19,19
1,25D1A5D72C59C493,classic_bike,2021-02-28 10:34:39,2021-02-28 11:00:01,19th St at Florida St,SF-O25-1,2nd St at Folsom St,SF-H29,37.760437,-122.410778,37.785318,-122.396434,member,3.037043e+06,1522.0,1995.429149,10,11
2,5D7DCF49E7C54D43,electric_bike,2021-02-14 13:44:57,2021-02-14 13:51:55,19th St at Florida St,SF-O25-1,Bryant St at 15th St,SF-M25,37.760437,-122.410778,37.767100,-122.410662,member,7.396375e+05,418.0,1769.467733,13,13
3,EF8B274C5CD4559E,electric_bike,2021-02-22 18:24:57,2021-02-22 18:50:10,Utah St at 24th St,SF-Q26,24th St at Bartlett St,SF-Q23-3,37.753243,-122.405496,37.752131,-122.419883,member,1.273815e+06,1513.0,841.913402,18,18
4,A896A2BE15365157,classic_bike,2021-02-22 13:16:00,2021-02-22 13:21:37,19th St at Florida St,SF-O25-1,16th St Mission BART Station 2,SF-N22-1B,37.760437,-122.410778,37.764854,-122.420129,member,9.587779e+05,337.0,2845.038145,13,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60274,6CC0C441607E4BAB,classic_bike,2021-02-18 06:02:39,2021-02-18 06:22:48,Fell St at Stanyan St,SF-K15-,17th & Folsom Street Park (17th St at Folsom St),SF-N23,37.771954,-122.453703,37.763720,-122.415400,member,3.496379e+06,1209.0,2891.959774,6,6
60275,ECC333B00108B2EB,classic_bike,2021-02-27 06:25:11,2021-02-27 06:40:34,Fell St at Stanyan St,SF-K15-,17th & Folsom Street Park (17th St at Folsom St),SF-N23,37.771954,-122.453703,37.763720,-122.415400,member,3.496379e+06,923.0,3788.059986,6,6
60276,63971FE6AFC02507,classic_bike,2021-02-25 06:25:20,2021-02-25 06:41:34,Fell St at Stanyan St,SF-K15-,17th & Folsom Street Park (17th St at Folsom St),SF-N23,37.771954,-122.453703,37.763720,-122.415400,member,3.496379e+06,974.0,3589.711876,6,6
60277,D343E2081AB58E27,classic_bike,2021-02-28 06:13:37,2021-02-28 06:31:12,Fell St at Stanyan St,SF-K15-,17th & Folsom Street Park (17th St at Folsom St),SF-N23,37.771954,-122.453703,37.763720,-122.415400,member,3.496379e+06,1055.0,3314.103665,6,6


In [5]:
elevation.loc['SF-M26']['SF-M25']

3.0

In [48]:
bikes['ele_diff']=0
for index,row in bikes.iterrows():
    try:
        bikes.loc[index,'ele_diff']=elevation.loc[row['start_station_id']][row['end_station_id']]
        print(bikes.loc[index,'ele_diff'])
    except KeyError:
        pass

bikes

7.0
8.0
6.0
8.0
2.0
-79.0
-55.0
23.0
16.0
7.0
-10.0
0.0
40.0
1.0
44.0
44.0
44.0
-4.0
7.0
7.0
7.0
7.0
-12.0
3.0
-38.0
-51.0
-51.0
3.0
-14.0
-14.0
-11.0
-51.0
-51.0
-47.0
-47.0
-76.0
-3.0
-2.0
3.0
-29.0
-29.0
-54.0
-54.0
-55.0
-55.0
16.0
16.0
17.0
-10.0
-18.0
-42.0
-20.0
-20.0
-3.0
-4.0
-4.0
-4.0
-4.0
-4.0
0.0
-51.0
13.0
-61.0
-61.0
-26.0
-26.0
-26.0
-26.0
-26.0
-9.0
-51.0
13.0
-10.0
15.0
-33.0
-33.0
-33.0
-8.0
10.0
59.0
49.0
54.0
31.0
31.0
42.0
10.0
-37.0
-12.0
-12.0
-12.0
-25.0
-25.0
-25.0
4.0
-2.0
1.0
1.0
1.0
0.0
-3.0
-28.0
-17.0
-45.0
22.0
-12.0
-4.0
-4.0
-12.0
-79.0
-12.0
-4.0
-29.0
-29.0
-29.0
-29.0
27.0
27.0
27.0
-15.0
-15.0
-15.0
27.0
27.0
27.0
12.0
6.0
6.0
6.0
6.0
6.0
-2.0
14.0
6.0
6.0
-42.0
11.0
-8.0
7.0
7.0
7.0
-69.0
-14.0
-9.0
-33.0
1.0
4.0
-7.0
8.0
8.0
-83.0
-58.0
-10.0
-83.0
-83.0
11.0
9.0
9.0
9.0
9.0
16.0
9.0
22.0
22.0
-25.0
6.0
-11.0
13.0
-21.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
19.0
13.0
13.0
1.0
1.0
1.0
-10.0
-20.0
-20.0
-20.0
-20.0
-20.0
-22.0
-11.0
1.0
33.0
6.0
40.0
40.0
40.

-3.0
14.0
14.0
-2.0
33.0
-6.0
-31.0
-3.0
-6.0
-6.0
46.0
-35.0
69.0
16.0
17.0
17.0
16.0
11.0
11.0
16.0
16.0
11.0
16.0
8.0
8.0
8.0
13.0
-17.0
12.0
12.0
-3.0
-12.0
7.0
2.0
-18.0
-69.0
-46.0
-69.0
-46.0
13.0
13.0
13.0
-8.0
6.0
6.0
6.0
-2.0
-2.0
14.0
9.0
-5.0
-7.0
12.0
12.0
10.0
12.0
-5.0
-7.0
-7.0
-7.0
-7.0
-5.0
-22.0
-24.0
-17.0
-18.0
-18.0
-18.0
-18.0
-18.0
-23.0
-18.0
-18.0
-10.0
-8.0
-10.0
-18.0
-18.0
-18.0
-18.0
-18.0
-18.0
-18.0
-18.0
-10.0
-18.0
-18.0
-25.0
-25.0
-25.0
-25.0
-20.0
-18.0
-8.0
11.0
-18.0
-18.0
-20.0
-18.0
-10.0
-18.0
-18.0
-18.0
-18.0
-10.0
-69.0
-44.0
-71.0
-71.0
11.0
11.0
11.0
11.0
9.0
-28.0
-26.0
10.0
8.0
-40.0
14.0
-26.0
-26.0
-26.0
-26.0
9.0
9.0
-4.0
-16.0
-4.0
-4.0
-4.0
20.0
10.0
-29.0
10.0
16.0
-2.0
-26.0
-2.0
-2.0
-26.0
-26.0
-17.0
-17.0
-17.0
-18.0
-59.0
-10.0
9.0
9.0
-18.0
-18.0
8.0
10.0
10.0
10.0
8.0
10.0
10.0
5.0
10.0
8.0
10.0
8.0
8.0
-7.0
-7.0
-5.0
-29.0
2.0
-26.0
-26.0
-17.0
13.0
13.0
1.0
-44.0
-44.0
-69.0
-69.0
-69.0
-69.0
-57.0
-18.0
-4.0
-4.0
-4.0
-12

0.0
0.0
-2.0
-2.0
3.0
-2.0
-2.0
-2.0
-2.0
-2.0
-39.0
3.0
-5.0
3.0
3.0
-2.0
51.0
-39.0
11.0
0.0
3.0
3.0
3.0
3.0
-5.0
-5.0
-3.0
-43.0
45.0
-3.0
-3.0
-39.0
14.0
27.0
-10.0
1.0
1.0
-16.0
-19.0
-19.0
-19.0
-19.0
-19.0
-19.0
-19.0
-19.0
-35.0
-12.0
-31.0
-39.0
-39.0
-39.0
-39.0
-39.0
-31.0
1.0
-3.0
-3.0
-65.0
-15.0
2.0
-3.0
26.0
-1.0
-17.0
22.0
1.0
1.0
4.0
-2.0
4.0
1.0
1.0
1.0
1.0
36.0
0.0
-6.0
0.0
-27.0
-69.0
-35.0
-35.0
-4.0
-35.0
-35.0
-46.0
-15.0
-15.0
-15.0
-15.0
-4.0
-15.0
-3.0
0.0
-5.0
8.0
-30.0
16.0
-16.0
-16.0
-16.0
34.0
-13.0
-82.0
34.0
34.0
34.0
-3.0
-3.0
-16.0
-3.0
-49.0
-35.0
-4.0
-77.0
-27.0
-35.0
3.0
-20.0
45.0
68.0
45.0
45.0
1.0
42.0
42.0
42.0
81.0
-82.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
1.0
1.0
1.0
0.0
0.0
0.0
0.0
49.0
84.0
84.0
42.0
42.0
42.0
42.0
42.0
42.0
42.0
42.0
42.0
42.0
-5.0
42.0
42.0
42.0
42.0
42.0
42.0
42.0
42.0
23.0
23.0
66.0
1.0
50.0
39.0
48.0
39.0
39.0
57.0
61.0
-3.0
53.0
53.0
53.0
53.0
53.0
104.0
75.0
-1.0
65.0
65.0
-1.0
51.0
71.0
-13.0
-6.0
-1.0
-6.0
-6.

1.0
1.0
1.0
1.0
1.0
3.0
3.0
3.0
-4.0
3.0
-33.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
2.0
-46.0
3.0
-19.0
-40.0
2.0
-2.0
3.0
3.0
3.0
2.0
2.0
-7.0
-29.0
5.0
-40.0
-8.0
-1.0
2.0
2.0
2.0
2.0
2.0
3.0
3.0
3.0
3.0
-4.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-31.0
-16.0
-1.0
-30.0
-1.0
-1.0
2.0
-8.0
23.0
-31.0
-11.0
-4.0
13.0
2.0
2.0
2.0
2.0
2.0
24.0
2.0
14.0
14.0
14.0
1.0
1.0
-7.0
-1.0
21.0
-1.0
-43.0
-17.0
0.0
-1.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
-6.0
-6.0
-6.0
-6.0
-6.0
-6.0
-6.0
-6.0
-6.0
-6.0
-6.0
-6.0
-6.0
-6.0
-6.0
-6.0
-6.0
3.0
3.0
-23.0
-32.0
-12.0
-1.0
21.0
-47.0
-9.0
-9.0
-31.0
-31.0
2.0
2.0
-30.0
-8.0
1.0
1.0
5.0
5.0
1.0
1.0
1.0
0.0
-32.0
22.0
-83.0
-4.0
-27.0
-9.0
-9.0
-9.0
15.0
-9.0
19.0
-3.0
-3.0
-3.0
-3.0
-16.0
-21.0
-39.0
-24.0
16.0
-3.0
-9.0
15.0
10.0
-33.0
8.0
-14.0
-14.0
-14.0
-13.0
-12.0
-12.0
-1.0
-1.0
-1.0
-1.0
0.0
-14.0
42.0
12.0
12.0
0.0
-4.0
33.0
42.0
11.0
-1.0
-1.0
-1.0
-1.0
-1.0
-14.0
-1.0
-71.0
-71

-45.0
-31.0
-6.0
-2.0
0.0
0.0
0.0
0.0
-6.0
-20.0
-6.0
-6.0
-6.0
-6.0
-6.0
-60.0
-60.0
-60.0
-60.0
-60.0
-60.0
-60.0
-60.0
-60.0
-4.0
-2.0
-2.0
-2.0
-2.0
-3.0
-5.0
31.0
31.0
-5.0
-5.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-70.0
-39.0
-39.0
-39.0
-39.0
-39.0
-39.0
1.0
-15.0
-42.0
-5.0
-32.0
49.0
1.0
1.0
-6.0
-31.0
-11.0
-45.0
-6.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-33.0
-19.0
-20.0
-18.0
-18.0
-18.0
-18.0
-18.0
-11.0
-11.0
-11.0
1.0
-33.0
-33.0
-33.0
-33.0
-33.0
-33.0
-33.0
-33.0
-33.0
-33.0
-33.0
-33.0
-33.0
-33.0
-33.0
-33.0
-31.0
0.0
-2.0
-10.0
-84.0
-12.0
1.0
1.0
-56.0
-9.0
-9.0
-20.0
-20.0
40.0
15.0
11.0
11.0
6.0
2.0
-18.0
2.0
9.0
9.0
9.0
29.0
-10.0
39.0
39.0
39.0
39.0
39.0
3.0
29.0
12.0
-18.0
-18.0
-18.0
-18.0
2.0
5.0
5.0
5.0
8.0
8.0
8.0
8.0
8.0
8.0
-6.0
13.0
13.0
12.0
37.0
35.0
-10.0
-10.0
7.0
7.0
-7.0
-7.0
-21.0
-7.0
-7.0
-7.0
-21.0
67.0
1.0
1.0
9.0
1.0
10.0
-4.0
-7.0
11.0
11.0
11.0
-12.0
-12.0
11.0
-12.0
-12.0
3.0
-28.0
-28.0
-28.0
-28.0
-28.0
-4.0
24.0
-28.0
-28.0
-28.0
-28.0
-28

...

-18.0
-18.0
-18.0
-18.0
-18.0
-18.0
-18.0
-18.0
-18.0
-18.0
-18.0
-18.0
-18.0
-18.0
-18.0
-18.0
-18.0
-18.0
-18.0
-18.0
-18.0
-2.0
-10.0
-18.0
-18.0
-9.0
-10.0
-10.0
-18.0
-18.0
-18.0
10.0
10.0
10.0
1.0
1.0
1.0
10.0
-57.0
10.0
8.0
-47.0
-47.0
-47.0
-47.0
-18.0
23.0
-28.0
53.0
-1.0
-1.0
-30.0
-39.0
-30.0
0.0
23.0
-28.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
11.0
-25.0
-25.0
-34.0
-3.0
-3.0
-3.0
-3.0
-3.0
10.0
2.0
-34.0
63.0
2.0
11.0
11.0
11.0
11.0
11.0
11.0
11.0
-34.0
1.0
1.0
1.0
1.0
10.0
2.0
1.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
13.0
-34.0
-34.0
-34.0
-34.0
-34.0
-34.0
-34.0
-34.0
-34.0
-34.0
-34.0
-34.0
-34.0
-34.0
-34.0
-34.0
1.0
1.0
1.0
63.0
-26.0
2.0
2.0
2.0
2.0
1.0
9.0
9.0
1.0
1.0
10.0
1.0
9.0
9.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
-2.0
-2.0
-2.0
-2.0
-2.0
7.0
7.0
16.0
-17.0
7.0
-17.0
-2.0
-2.0
-2.0
7.0
7.0
7.0
-2.0
6.0
-16.0
-2.0
7.0
6.0
-23.0
-2.0
-16.0
6.0
-2.0
19.0
-2.0
7.0
7.0
-22.0
-16.0
31.0
-31.0
-16.0
-17.0
-17.0
-17.0
21.0
5.0
5.0
5.0
32.0
21.0
9.0
9.0
9.

10.0
2.0
2.0
2.0
2.0
2.0
2.0
-12.0
1.0
-23.0
-17.0
7.0
7.0
7.0
22.0
-8.0
1.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
1.0
1.0
-21.0
-13.0
-13.0
-4.0
-21.0
-21.0
-21.0
10.0
10.0
10.0
10.0
63.0
63.0
-3.0
1.0
11.0
11.0
11.0
11.0
-24.0
-24.0
2.0
-16.0
13.0
1.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
0.0
1.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
1.0
54.0
1.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
-8.0
14.0
-48.0
-48.0
-48.0
-4.0
-13.0
-13.0
-64.0
2.0
2.0
49.0
8.0
-64.0
11.0
-30.0
-20.0
-17.0
11.0
-64.0
-76.0
-23.0
-77.0
-64.0
-9.0
-9.0
-9.0
-9.0
-9.0
12.0
-9.0
-14.0
53.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
4.0
4.0
4.0
4.0
4.0
-12.0
-12.0
-12.0
-12.0
-12.0
-3.0
11.0
10.0
51.0
2.0
-12.0
-12.0
4.0
-3.0
-3.0
2.0
5.0
4.0
5.0
5.0
5.0
5.0
-4.0
-4.0
-4.0
-4.0
-4.0
-4.0
-4.0
2.0
2.0
2.0
-12.0
-12.0
-12.0
-4.0
5.0
4.0
-12.0
-11.0
-11.0
-11.0
-11.0
-11.0
-11.0
-11.0
-6.0
2.0
-5.0
3.0
-46.0
-4

13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
13.0
4.0
-8.0
13.0
13.0
13.0
-45.0
-45.0
-45.0
-45.0
13.0
13.0
-15.0
-15.0
-15.0
-15.0
-15.0
13.0
13.0
-45.0
2.0
10.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
34.0
34.0
10.0
1.0
-52.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
-11.0
9.0
18.0
10.0
-3.0
-11.0
18.0
18.0
18.0
18.0
34.0
34.0
34.0
34.0
26.0
34.0
34.0
34.0
-28.0
34.0
-28.0
34.0
34.0
34.0
34.0
34.0
26.0
-24.0
-24.0
7.0
-3.0
15.0
14.0
76.0
7.0
0.0
-58.0
-58.0
-58.0
30.0
1.0
1.0
-2.0
1.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-2.0
-1.0
12.0
-1.0
-10.0
-10.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-9.0
-22.0
-10.0
-59.0
-59.0
-3.0
0.0
7.0
4.0
7.0
-55.0
-55.0
-55.0
-55.0
-55.0
-55.0
-14.0
-3.0
-3.0
-3.0
-3.0
-3.0
-3.0
-3.0
-3.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
7.0
-

9.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
9.0
-12.0
22.0
-53.0
14.0
-53.0
-9.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
15.0
-46.0
-46.0
-7.0
15.0
-46.0
-46.0
0.0
3.0
12.0
12.0
12.0
12.0
12.0
3.0
3.0
3.0
3.0
4.0
3.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
12.0
3.0
12.0
12.0
3.0
12.0
12.0
12.0
1.0
3.0
10.0
12.0
-9.0
12.0
10.0
10.0
4.0
10.0
12.0
10.0
1.0
62.0
-5.0
-2.0
-57.0
5.0
5.0
5.0
5.0
-3.0
-9.0
21.0
-60.0
-60.0
8.0
-2.0
-60.0
-60.0
58.0
-9.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
-9.0
2.0
12.0
23.0
-24.0
-24.0
7.0
2.0
-2.0
-13.0
34.0
34.0
34.0
34.0
34.0
-7.0
2.0
2.0
2.0
2.0
2.0
-7.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
-2.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
-11.0
1.0
34.0
-10.0
43.0
31.0
18.0
18.0
18.0
18.0
18.0
18.0
18.0
-4.0
-4.0
25.0
34.0
20.0
35.0
31.0
9.0
9.0
9.0
20.0
34.0
13.0
-2.0
26.0
34.0
25.0
25.0
34.0
34.0
-28.0
-28.0
13.0
-24.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
34.0
31.0
31.0
-1.0
-1.0
-1.0
3

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,displacement,duration,velocity,start_hour,end_hour,ele_diff,type_num
0,072879C2ED883CF1,classic_bike,2021-02-12 19:08:22,2021-02-12 19:15:02,Townsend St at 5th St,SF-K28,Mississippi St at 17th St,SF-N29,37.775214,-122.397477,37.764793,-122.394796,member,1.180490e+06,400.0,2951.224312,19,19,7.0,1
1,25D1A5D72C59C493,classic_bike,2021-02-28 10:34:39,2021-02-28 11:00:01,19th St at Florida St,SF-O25-1,2nd St at Folsom St,SF-H29,37.760437,-122.410778,37.785318,-122.396434,member,3.037043e+06,1522.0,1995.429149,10,11,8.0,1
2,5D7DCF49E7C54D43,electric_bike,2021-02-14 13:44:57,2021-02-14 13:51:55,19th St at Florida St,SF-O25-1,Bryant St at 15th St,SF-M25,37.760437,-122.410778,37.767100,-122.410662,member,7.396375e+05,418.0,1769.467733,13,13,6.0,0
3,EF8B274C5CD4559E,electric_bike,2021-02-22 18:24:57,2021-02-22 18:50:10,Utah St at 24th St,SF-Q26,24th St at Bartlett St,SF-Q23-3,37.753243,-122.405496,37.752131,-122.419883,member,1.273815e+06,1513.0,841.913402,18,18,8.0,0
4,A896A2BE15365157,classic_bike,2021-02-22 13:16:00,2021-02-22 13:21:37,19th St at Florida St,SF-O25-1,16th St Mission BART Station 2,SF-N22-1B,37.760437,-122.410778,37.764854,-122.420129,member,9.587779e+05,337.0,2845.038145,13,13,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60274,6CC0C441607E4BAB,classic_bike,2021-02-18 06:02:39,2021-02-18 06:22:48,Fell St at Stanyan St,SF-K15-,17th & Folsom Street Park (17th St at Folsom St),SF-N23,37.771954,-122.453703,37.763720,-122.415400,member,3.496379e+06,1209.0,2891.959774,6,6,-71.0,1
60275,ECC333B00108B2EB,classic_bike,2021-02-27 06:25:11,2021-02-27 06:40:34,Fell St at Stanyan St,SF-K15-,17th & Folsom Street Park (17th St at Folsom St),SF-N23,37.771954,-122.453703,37.763720,-122.415400,member,3.496379e+06,923.0,3788.059986,6,6,-71.0,1
60276,63971FE6AFC02507,classic_bike,2021-02-25 06:25:20,2021-02-25 06:41:34,Fell St at Stanyan St,SF-K15-,17th & Folsom Street Park (17th St at Folsom St),SF-N23,37.771954,-122.453703,37.763720,-122.415400,member,3.496379e+06,974.0,3589.711876,6,6,-71.0,1
60277,D343E2081AB58E27,classic_bike,2021-02-28 06:13:37,2021-02-28 06:31:12,Fell St at Stanyan St,SF-K15-,17th & Folsom Street Park (17th St at Folsom St),SF-N23,37.771954,-122.453703,37.763720,-122.415400,member,3.496379e+06,1055.0,3314.103665,6,6,-71.0,1


In [49]:
bikes['type_num']=0
bikes['type_num']=[1 if x=='classic_bike' else 0 for x in bikes['rideable_type']]
bikes

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,displacement,duration,velocity,start_hour,end_hour,ele_diff,type_num
0,072879C2ED883CF1,classic_bike,2021-02-12 19:08:22,2021-02-12 19:15:02,Townsend St at 5th St,SF-K28,Mississippi St at 17th St,SF-N29,37.775214,-122.397477,37.764793,-122.394796,member,1.180490e+06,400.0,2951.224312,19,19,7.0,1
1,25D1A5D72C59C493,classic_bike,2021-02-28 10:34:39,2021-02-28 11:00:01,19th St at Florida St,SF-O25-1,2nd St at Folsom St,SF-H29,37.760437,-122.410778,37.785318,-122.396434,member,3.037043e+06,1522.0,1995.429149,10,11,8.0,1
2,5D7DCF49E7C54D43,electric_bike,2021-02-14 13:44:57,2021-02-14 13:51:55,19th St at Florida St,SF-O25-1,Bryant St at 15th St,SF-M25,37.760437,-122.410778,37.767100,-122.410662,member,7.396375e+05,418.0,1769.467733,13,13,6.0,0
3,EF8B274C5CD4559E,electric_bike,2021-02-22 18:24:57,2021-02-22 18:50:10,Utah St at 24th St,SF-Q26,24th St at Bartlett St,SF-Q23-3,37.753243,-122.405496,37.752131,-122.419883,member,1.273815e+06,1513.0,841.913402,18,18,8.0,0
4,A896A2BE15365157,classic_bike,2021-02-22 13:16:00,2021-02-22 13:21:37,19th St at Florida St,SF-O25-1,16th St Mission BART Station 2,SF-N22-1B,37.760437,-122.410778,37.764854,-122.420129,member,9.587779e+05,337.0,2845.038145,13,13,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60274,6CC0C441607E4BAB,classic_bike,2021-02-18 06:02:39,2021-02-18 06:22:48,Fell St at Stanyan St,SF-K15-,17th & Folsom Street Park (17th St at Folsom St),SF-N23,37.771954,-122.453703,37.763720,-122.415400,member,3.496379e+06,1209.0,2891.959774,6,6,-71.0,1
60275,ECC333B00108B2EB,classic_bike,2021-02-27 06:25:11,2021-02-27 06:40:34,Fell St at Stanyan St,SF-K15-,17th & Folsom Street Park (17th St at Folsom St),SF-N23,37.771954,-122.453703,37.763720,-122.415400,member,3.496379e+06,923.0,3788.059986,6,6,-71.0,1
60276,63971FE6AFC02507,classic_bike,2021-02-25 06:25:20,2021-02-25 06:41:34,Fell St at Stanyan St,SF-K15-,17th & Folsom Street Park (17th St at Folsom St),SF-N23,37.771954,-122.453703,37.763720,-122.415400,member,3.496379e+06,974.0,3589.711876,6,6,-71.0,1
60277,D343E2081AB58E27,classic_bike,2021-02-28 06:13:37,2021-02-28 06:31:12,Fell St at Stanyan St,SF-K15-,17th & Folsom Street Park (17th St at Folsom St),SF-N23,37.771954,-122.453703,37.763720,-122.415400,member,3.496379e+06,1055.0,3314.103665,6,6,-71.0,1


Create a Column for quantifiable predction, if bike is classic then 1 else 0

In [59]:
bikes['ele_diff'].min()

-108.0

In [63]:

for i in range(-100,80):
    X=pd.DataFrame(bikes[bikes['ele_diff']>i]['ele_diff'])
    Y=pd.DataFrame(bikes[bikes['ele_diff']>i]['type_num'])
    X_scale = (X-X.mean())/X.std()
    lr = LogisticRegression(fit_intercept=True)
    X_train, X_test, Y_train, Y_test = train_test_split(X_scale, Y, test_size=0.4,random_state=42)
    lr.fit(X_train,Y_train)
    score = lr.score(X_test, Y_test)
    print('for elevation',i,'the score is ',score)

C:\Users\kshit\anaconda3\envs\Kshitij\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\kshit\anaconda3\envs\Kshitij\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\kshit\anaconda3\envs\Kshitij\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\kshit\anaconda3\envs\Kshitij\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change 

for elevation -100 the score is  0.5324648384018587
for elevation -99 the score is  0.5340220728570243
for elevation -98 the score is  0.5321550078831633
for elevation -97 the score is  0.5310153105680262
for elevation -96 the score is  0.5353334163243288
for elevation -95 the score is  0.5353334163243288


C:\Users\kshit\anaconda3\envs\Kshitij\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\kshit\anaconda3\envs\Kshitij\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\kshit\anaconda3\envs\Kshitij\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\kshit\anaconda3\envs\Kshitij\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change 

for elevation -94 the score is  0.5319307855097722
for elevation -93 the score is  0.5338617312640053
for elevation -92 the score is  0.5355631172711428
for elevation -91 the score is  0.535438625612084
for elevation -90 the score is  0.535438625612084
for elevation -89 the score is  0.5323484251151596


C:\Users\kshit\anaconda3\envs\Kshitij\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\kshit\anaconda3\envs\Kshitij\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\kshit\anaconda3\envs\Kshitij\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\kshit\anaconda3\envs\Kshitij\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change 

...

C:\Users\kshit\anaconda3\envs\Kshitij\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\kshit\anaconda3\envs\Kshitij\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\kshit\anaconda3\envs\Kshitij\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\kshit\anaconda3\envs\Kshitij\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change 

for elevation 45 the score is  0.8568047337278106
for elevation 46 the score is  0.8675
for elevation 47 the score is  0.8637566137566137
for elevation 48 the score is  0.8585434173669467
for elevation 49 the score is  0.8620689655172413
for elevation 50 the score is  0.8805031446540881
for elevation 51 the score is  0.8739635157545605
for elevation 52 the score is  0.8627787307032591
for elevation 53 the score is  0.8608058608058609
for elevation 54 the score is  0.8813229571984436
for elevation 55 the score is  0.8677685950413223
for elevation 56 the score is  0.8793859649122807
for elevation 57 the score is  0.8644859813084113
for elevation 58 the score is  0.8847117794486216
for elevation 59 the score is  0.8909574468085106
for elevation 60 the score is  0.884393063583815
for elevation 61 the score is  0.8975155279503105
for elevation 62 the score is  0.8668831168831169


C:\Users\kshit\anaconda3\envs\Kshitij\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\kshit\anaconda3\envs\Kshitij\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\kshit\anaconda3\envs\Kshitij\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\kshit\anaconda3\envs\Kshitij\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change 

for elevation 63 the score is  0.863013698630137
for elevation 64 the score is  0.8787878787878788
for elevation 65 the score is  0.8849206349206349
for elevation 66 the score is  0.9122807017543859
for elevation 67 the score is  0.909952606635071
for elevation 68 the score is  0.8877005347593583
for elevation 69 the score is  0.8780487804878049
for elevation 70 the score is  0.881578947368421
for elevation 71 the score is  0.8857142857142857
for elevation 72 the score is  0.9212598425196851
for elevation 73 the score is  0.9083333333333333
for elevation 74 the score is  0.9351851851851852
for elevation 75 the score is  0.9381443298969072
for elevation 76 the score is  0.9294117647058824
for elevation 77 the score is  0.9342105263157895
for elevation 78 the score is  0.9142857142857143
for elevation 79 the score is  0.9508196721311475


C:\Users\kshit\anaconda3\envs\Kshitij\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\kshit\anaconda3\envs\Kshitij\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\kshit\anaconda3\envs\Kshitij\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Checking accuracy for displacement.

In [37]:
X

,ele_diff
7,23.0
12,40.0
14,44.0
15,44.0
16,44.0
...,...
59949,25.0
59951,46.0
59952,46.0
59953,32.0


In [38]:
Y

,type_num
7,0
12,1
14,0
15,0
16,0
...,...
59949,0
59951,0
59952,0
59953,0


In [39]:
X_scale = (X-X.mean())/X.std()

In [40]:
lr = LogisticRegression(fit_intercept=True)

In [41]:
X_train, X_test, Y_train, Y_test = train_test_split(X_scale, Y, test_size=0.4,random_state=42) 

Randomly divide into train and test dataset

In [42]:
lr.fit(X_train,Y_train)

C:\Users\kshit\anaconda3\envs\Kshitij\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression()

In [43]:
preds=lr.predict(X_test)
preds

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [44]:
score = lr.score(X_test, Y_test)
score

0.7923076923076923

Elevation is not a good predictor at all.